In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import bartz
from pyinstrument import Profiler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from bart_playground import *
import arviz as az

INFO:arviz.preview:arviz_base not installed
INFO:arviz.preview:arviz_stats not installed
INFO:arviz.preview:arviz_plots not installed


In [3]:
generator = DataGenerator(n_samples=160, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
preprocessor = DefaultPreprocessor()
data = preprocessor.fit_transform(X_train, y_train)

In [5]:
n_trees = 100
rng = np.random.default_rng(42)
random_trees_uniform = create_random_init_trees(
    n_trees=n_trees,
    dataX=data.X,
    possible_thresholds=preprocessor.thresholds,
    generator=rng
)

In [6]:
random_trees_uniform[0]

Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.7601702       nan       nan       nan       nan       nan       nan
       nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=[120  92  28   0   0   0   0   0])

In [7]:
proposal_probs = {"grow": 0.25, "prune": 0.25, "change": 0.4, "swap": 0.1}
bart = DefaultBART(ndpost=200, nskip=0, n_trees=100, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)

Iterations: 100%|██████████| 200/200 [00:01<00:00, 163.10it/s]


In [8]:
bart.trace[0].trees

[Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),

In [9]:
y_pred = bart.predict(X_test)
mean_squared_error(y_test, y_pred)

0.023285599025382794

In [10]:
bart_init = DefaultBART(ndpost=200, nskip=0, n_trees=100, proposal_probs=proposal_probs, 
                   init_trees=random_trees_uniform)
bart_init.fit(X_train, y_train)

Iterations: 100%|██████████| 200/200 [00:00<00:00, 241.92it/s]


In [11]:
bart_init.trace[0].trees

[Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.7601702       nan       nan       nan       nan       nan       nan
        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=[ 1 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.4408347       nan       nan       nan       nan       nan       nan
        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.83016026        nan        nan        nan        nan        nan
         nan        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.66302365        nan        nan        nan        nan        nan
         nan        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.09292644        nan        nan        nan        nan        nan
         nan        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=

In [12]:
y_pred = bart_init.predict(X_test)
mean_squared_error(y_test, y_pred)

0.023152337671197197

In [13]:
proposal_probs = {"multi_grow": 0.25, "multi_prune": 0.25, "multi_change": 0.4, "multi_swap": 0.1}
bart_mtmh = MultiBART(ndpost=200, nskip=0, n_trees=100, proposal_probs=proposal_probs, multi_tries=10)
bart_mtmh.fit(X_train, y_train)

Iterations: 100%|██████████| 200/200 [00:06<00:00, 28.67it/s]


In [14]:
bart_mtmh.trace[0].trees

[Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),
 Tree(vars=[-1 -2 -2 -2 -2 -2 -2 -2], thresholds=[nan nan nan nan nan nan nan nan], leaf_vals=[ 0. nan nan nan nan nan nan nan], n_vals=None),

In [15]:
y_pred = bart_mtmh.predict(X_test)
mean_squared_error(y_test, y_pred)

0.022599835171959966

In [16]:
bart_mtmh_init = MultiBART(ndpost=200, nskip=0, n_trees=100, proposal_probs=proposal_probs, 
                           multi_tries=10, init_trees=random_trees_uniform)
bart_mtmh_init.fit(X_train, y_train)

Iterations: 100%|██████████| 200/200 [00:07<00:00, 27.61it/s]


In [17]:
bart_mtmh_init.trace[0].trees

[Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.7601702       nan       nan       nan       nan       nan       nan
        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=[ 1 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.4408347       nan       nan       nan       nan       nan       nan
        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.83016026        nan        nan        nan        nan        nan
         nan        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.66302365        nan        nan        nan        nan        nan
         nan        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=[ 0 -1 -1 -2 -2 -2 -2 -2], thresholds=[0.09292644        nan        nan        nan        nan        nan
         nan        nan], leaf_vals=[nan  0.  0. nan nan nan nan nan], n_vals=None),
 Tree(vars=

In [18]:
y_pred = bart_mtmh_init.predict(X_test)
mean_squared_error(y_test, y_pred)

0.023033653428438723